<a href="https://colab.research.google.com/github/maddran/lordstanleyscup/blob/main/code/LSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#@title Clone git repo and get helpers

!rm -r lordstanleyscup/
!git clone https://github.com/maddran/lordstanleyscup

from lordstanleyscup.code.helpers import *

Cloning into 'lordstanleyscup'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 70 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (70/70), done.


In [32]:
SEASON = 2022 #@param {type:"integer"}
SCWINNER = "COL" #@param {type:"string"}




In [33]:
#@title Get schedule and draft results

sched_df = get_schedule(SEASON)

draft_data_path = f"/content/lordstanleyscup/draft_data/Stanley Cup - {str(SEASON)[-2:]}_{str(SEASON+1)[-2:]} - Draft.csv"
draft_df = pd.read_csv(draft_data_path)
draft_df.columns = ['round', 'pick', 'owner', 'team']

draft_LU = dict(zip(draft_df.team, draft_df.owner))

score_df = sched_df

score_df['cupGame'] = 0
score_df['cupTeam'] = None

In [34]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def next_cup_game(df, cupteam):
  return min((df[df.cupTeam.isnull()].homeTeam == cupteam).idxmax(),
    (df[df.cupTeam.isnull()].awayTeam == cupteam).idxmax())

def get_winner(row):
  if row.homeScore > row.awayScore:
    return row.homeTeam
  elif row.homeScore < row.awayScore:
    return row.awayTeam
  else:
    return None

def score_season(df, cupteam = SCWINNER):

  if df.cupTeam.last_valid_index() is None:
    last_idx = 0
  else:
    last_idx = df.cupTeam.last_valid_index()
    cupteam = df.loc[last_idx,'cupTeam']

  next_idx = next_cup_game(df, cupteam)
  if(last_idx != 0 and next_idx <= last_idx+1):
    return

  df.loc[next_idx, 'cupGame'] = 1
  df.loc[last_idx:next_idx-1, 'cupTeam'] = cupteam
  cupteam = get_winner(df.loc[next_idx])
  df.loc[next_idx, 'cupTeam'] = cupteam

  score_season(df, cupteam)


In [35]:
score_season(score_df, SCWINNER)

for p in ['home', 'away', 'cup']:
  score_df[f'{p}Owner'] = score_df[f'{p}Team'].apply(lambda x: draft_LU.get(x)).fillna('')

In [36]:
#@title Total points and cup games

points_df = score_df[(score_df.cupGame == 1) & ((score_df.homeScore + score_df.awayScore) != 0)]

points_summary = points_df.groupby(['cupOwner']).cupGame.sum().reset_index()
points_summary.columns = ['owner', 'points']

cup_games_summary_home = points_df.groupby(['homeOwner','homeTeam']).cupGame.sum().reset_index()
cup_games_summary_home.columns = ['owner', 'team', 'cupGames']
cup_games_summary_away = points_df.groupby(['awayOwner','awayTeam']).cupGame.sum().reset_index()
cup_games_summary_away.columns = ['owner', 'team', 'cupGames']

cup_games_summary = pd.concat([cup_games_summary_away, cup_games_summary_home]).groupby(['owner']).cupGames.sum().reset_index()

full_summary = cup_games_summary.merge(points_summary, how = 'outer').fillna(0).sort_values(by=['points', 'cupGames'], ascending=[False,True])
full_summary

,owner,cupGames,points
0,A,5,3.0
1,E,4,2.0
2,M,1,0.0


In [42]:
#@title Point snake

snake = points_df.groupby(['gameDate','cupOwner','cupTeam']).cupGame.sum().groupby(level=1).cumsum().reset_index()
snake.columns = ['gameDate', 'owner', 'cupTeam', 'points']
snake = snake.pivot(index='gameDate', columns='owner', values='points')
snake.index = pd.to_datetime(snake.index)

snake.loc[min(snake.index) - timedelta(days=1)] = 0
snake.sort_index(inplace=True)

snake.loc[max(snake.index) + timedelta(days=1)] = snake.loc[max(snake.index), :]
snake.sort_index(inplace=True)

snake.iloc[0].fillna(0,inplace=True)
snake.ffill(axis = 0, inplace=True)

snake

owner,A,E
gameDate,,
2022-10-11,0.0,0.0
2022-10-12,0.0,1.0
2022-10-13,1.0,1.0
2022-10-15,2.0,1.0
2022-10-18,3.0,1.0
2022-10-20,3.0,2.0
2022-10-21,3.0,2.0


In [47]:
#@title Plot snake

import plotly.express as px

fig = px.line(snake.reset_index(), 
              x='gameDate', y=snake.columns, 
              template = 'plotly_dark', width=800, height=400)
fig.update_traces(mode="lines", line_shape="hv", line=dict(width=7))
fig.show()

In [39]:
#@title Current cup team

currentCupTeam = score_df.loc[score_df.cupTeam.last_valid_index(), 'cupTeam']
currentCupTeam

'BUF'

In [40]:
#@title Next cup game

next_cup_idx = next_cup_game(score_df,currentCupTeam)
score_df.loc[next_cup_idx,:]

gameDate     20221022
homeTeam          VAN
awayTeam          BUF
homeScore           0
awayScore           0
cupGame             1
cupTeam          None
homeOwner           A
awayOwner           E
cupOwner             
Name: 83, dtype: object

In [41]:
#@title Hypothetical next cup game

for winner in ['home', 'away']:
  hypo_df = score_df.iloc[next_cup_idx:,:].reset_index(drop=True)
  hypo_df.loc[0, f'{winner}Score'] = 1
  hypo_df.loc[0, 'cupTeam'] = hypo_df.loc[0, f'{winner}Team']
  hypo_df.loc[0, 'cupOwner'] = hypo_df.loc[0, f'{winner}Owner']

  hypo_team = hypo_df.loc[0, 'cupTeam']
  hypo_owner = hypo_df.loc[0, 'cupOwner']
  
  score_season(hypo_df, hypo_team)

  hypo_next_cup_idx = next_cup_game(hypo_df,hypo_team)

  print(f"\nIf {hypo_owner} wins:\n")
  print(hypo_df.loc[hypo_next_cup_idx,:])






If A wins:

gameDate     20221024
homeTeam          VAN
awayTeam          CAR
homeScore           0
awayScore           0
cupGame             1
cupTeam          None
homeOwner           A
awayOwner           A
cupOwner             
Name: 13, dtype: object

If E wins:

gameDate     20221025
homeTeam          SEA
awayTeam          BUF
homeScore           0
awayScore           0
cupGame             1
cupTeam          None
homeOwner           R
awayOwner           E
cupOwner             
Name: 21, dtype: object
